# Tutorial02 - Add transform keys

## Overview

🚀 By the last tutorial, you have learned the concept of ``Actor``, ``Level`` and ``Sequence``. In this tutorial, you will learn how to add transform keys to an ``Actor`` in order to render multiple poses of it. By the end of this tutorial, you will be able to:
- Open Blender/Unreal Engine by XRFeitoria
- Import meshes as the ``Actor``s in the ``Level``
- Set ``Actor``s' scale in the ``Level``
- Create a `Sequence` for rendering and adding transform keys to `Actor`s
- Add a camera in the `Sequence`
- Render and save images

## 1. Preparation

Install the following packages that will be used in this tutorial:

In [ ]:
%pip install objaverse
%pip install scipy

Then, similar as Tutorial 01, specify your engine path and initialize XRFeitoria.

In [ ]:
import xrfeitoria as xf

In [ ]:
# Replace with your executable path

# Blender
# engine_exec_path = 'D:/Program Files/Blender Foundation/Blender 3.3/blender.exe'

# Unreal Engine
engine_exec_path = 'D:/Program Files/Epic Games/UE_5.1/Engine/Binaries/Win64/UnrealEditor-Cmd.exe'

In [ ]:
from pathlib import Path

exec_path_stem = Path(engine_exec_path).stem.lower()
if 'blender' in exec_path_stem:
    # Open Blender
    render_engine = 'blender'
    xf_runner = xf.init_blender(exec_path=engine_exec_path, background=False, new_process=True, replace_plugin=True, dev_plugin=True)
elif 'unreal' in exec_path_stem:
    # Unreal Engine requires a project to be opened
    # Here we use a sample project, which is downloaded from the following link
    # You can also prepare your own project
    import shutil
    from xrfeitoria.utils.downloader import download
    unreal_project_zip = download(url='https://http://file.bj.zoe.sensetime.com/resources/meihaiyi/xrfeitoria/unreal_project/UE_sample.zip', 
                                    dst_dir="./tutorial02/assets/")
    shutil.unpack_archive(filename=unreal_project_zip, extract_dir='./tutorial02/assets/')

    # Open Unreal Engine
    render_engine = 'unreal'
    xf_runner = xf.init_unreal(exec_path=engine_exec_path, 
                                background=False, 
                                new_process=True, 
                                project_path='./tutorial02/assets/UE_sample/UE_sample.uproject',
                                replace_plugin=True, dev_plugin=True)

✨ Now you can see a new Blender/Unreal Engine process has started.

## 3. Import meshes

Download the meshes from [Objaverse](https://objaverse.allenai.org/objaverse-1.0/).

In [ ]:

import objaverse
objects = objaverse.load_objects(
    uids=['eb0807309530496aaab9dcff67bf5c31',
          'c55eff0309a14cf09423d238900cc7c2',
          'd0e07b22f1d54b968943e7a896235a65',
          'b4065dd5ce9d46be90db3e1f3e4b9cc1',
          '0176be079c2449e7aaebfb652910a854',
          'f130ebeb60f24ed8bd3714a7ed3ba280',
          '289a2221178843a78ad433705555e16a',
          'b7f7ab9bf7244c3a8851bae3fb0bf741',
        ],
    download_processes=1
)

Import the meshes to create ``Actor`` instances in the ``Level``.

In [ ]:

actors = []
for _, file_path in objects.items():
    actor = xf_runner.Actor.import_from_file(file_path=file_path)
    actors.append(actor)

Switch to the engine window, and you can see the meshes has been imported.

Then, we adjust the scale of the Actors to make their sizes equal to 0.2m.

In [ ]:
actor_size = 0.2
for actor in actors:
    actor.scale = (actor_size / max(actor.dimensions), ) * 3

If you use ``Unreal Engine``, the ``Level`` should be saved after you modify it.

In [ ]:
# save the level
if render_engine == 'unreal':
    xf_runner.utils.save_current_level()   

## 4. Add a sequence for rendering and adding transform keys

``Sequence`` is a multifunctional class in XRFeitoria. It can be used for:
- rendering
- adding transform keys
- grouping different objects.

Here, we use it for ``rendering`` and ``adding transform keys``.

>``Transform keys`` record the tranformation (location, rotation and scale) of an ``Actor`` at specific frames, and the tranformation between two adjacent keys will be interpolated by the specified interpolation method. By adding transform keys, you can render multiple poses of an ``Actor``.
>In ``XRFeitoria``, transform keys are always stored in a list, and the members of the list are [``SequenceTransformKey``](https://xrfeitoria.readthedocs.io/en/latest/apis/generated/xrfeitoria.data_structure.models.SequenceTransformKey.html) object.

Firstly, we randomly generate some transform keys for each actors.

In [ ]:
import random
from loguru import logger
from scipy.stats import qmc
from xrfeitoria.data_structure.models import SequenceTransformKey as SeqTransKey

# Set the number of frames
frame_num = 10

# Use a dictionary to store the transform keys of all actors
all_actors_transform_keys = {actor: [] for actor in actors}

# Iterate over all frames
for i in range(frame_num):

    # Generate random locations by Poisson Disk Sampling
    # The minimum distance between two actors is set to be `actor_size` we defined before
    posson_engine = qmc.PoissonDisk(d=2, radius=actor_size)
    sample_location = posson_engine.random(len(actors))
    
    # Set the transform keys for each actor
    for actor_idx, actor in enumerate(actors):
        actor_scale = actor.scale
        
        # Get the location from the samples generated by Poisson Disk Sampling
        random_location = (sample_location[actor_idx][0], 
                           0.0, 
                           sample_location[actor_idx][1])
        
        # Generate random rotations
        random_rotation = (random.random() * 360.0,
                           random.random() * 360.0,
                           random.random() * 360.0)
        
        # Generate random scales
        scale = random.uniform(0.8, 1.0)
        random_scale = (scale * actor_scale[0],
                        scale * actor_scale[1],
                        scale * actor_scale[2])
        
        # Save the transform keys
        all_actors_transform_keys[actor].append(
            SeqTransKey(
                frame=i,
                location=random_location,
                rotation=random_rotation,
                scale=random_scale,
                interpolation='AUTO',
            )            
        )
    logger.info(f'Generated transform keys of frame {i}.')

Then we create a ``Sequence`` to apply the transform keys to the actors and render the images.

In [ ]:
from xrfeitoria.data_structure.models import RenderPass

# Use the `with` statement to create a sequence, and it will be automatically close the sequence after the code block is executed.
# The argument `seq_length` controls the number of frames to be rendered. 
sequence_name = 'MySequence'
with xf_runner.Sequence.new(seq_name=sequence_name, seq_length=frame_num) as seq:
    ##############################
    ##### Add transform keys #####
    ##############################

    # The function `use_actor_with_keys` sets transform keys for the actor in the sequence.
    # The transform keys are only stored in the sequence.
    # When the sequence is closed, the actor will be restored to its original state(without transform keys).
    for actor, keys in all_actors_transform_keys.items():
        seq.use_actor_with_keys(actor=actor, transform_keys=keys)

    #####################
    ##### Rendering #####
    #####################

    # Add a camera and make it look at the specified location
    camera_location = (0.5, -3.0, 0.5)
    camera_rotation = xf_runner.utils.get_rotation_to_look_at(location=camera_location, target=(0.5, 0.0, 0.5))
    camera = seq.spawn_camera(location=camera_location, rotation=camera_rotation, fov=45)

    # Add a render job to renderer
    # In render job, you can specify the output path, resolution, render passes, etc.
    # The ``output_path`` is the path to save the rendered data.
    # The ``resolution`` is the resolution of the rendered image.
    # The ``render_passes`` define what kind of data you want to render, such as img, depth, normal, etc.
    # and what kind of format you want to save, such as png, jpg, exr, etc.
    seq.add_to_renderer(
        output_path=f'./tutorial02/outputs/{render_engine}/',
        resolution=(1280, 720),
        render_passes=[RenderPass('img', 'png'),
                       RenderPass('mask', 'exr'),
                       RenderPass('normal', 'exr'),
                       RenderPass('diffuse', 'exr'),
                       RenderPass('depth', 'exr'),
                       RenderPass('flow', 'exr')],
    )

## 5. Render

The following code renders all the render jobs and save the images to the ``output_path`` set in ``seq.add_to_renderer`` above.

In [ ]:
# Render
xf_runner.render()

Check the ``output_path``, and you can see the rendered images and their annotations. Visualize the images and annotations by the following code.

In [ ]:
import matplotlib.pyplot as plt
from xrfeitoria.utils.reader import XRFeitoriaReader

xf_viwer = XRFeitoriaReader(sequence_dir=f'./tutorial02/outputs/{render_engine}/{sequence_name}/')

for i in range(frame_num):
    img = xf_viwer.get_img(camera_name=camera.name, frame=i)
    mask = xf_viwer.get_mask(camera_name=camera.name, frame=i)
    normal = xf_viwer.get_normal(camera_name=camera.name, frame=i)
    diffuse = xf_viwer.get_diffuse(camera_name=camera.name, frame=i)
    depth = xf_viwer.get_depth(camera_name=camera.name, frame=i)
    flow = xf_viwer.get_flow(camera_name=camera.name, frame=i)

    plt.figure(figsize=(20, 20))
    plt.title(f'Frame {i}')

    plt.subplot(1, 6, 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title('img')

    plt.subplot(1, 6, 2)
    plt.imshow(mask)
    plt.axis('off')
    plt.title('mask')

    plt.subplot(1, 6, 3)
    plt.imshow(normal)
    plt.axis('off')
    plt.title('normal')

    plt.subplot(1, 6, 4)
    plt.imshow(diffuse)
    plt.axis('off')
    plt.title('diffuse')

    plt.subplot(1, 6, 5)
    plt.imshow(depth)
    plt.axis('off')
    plt.title('depth')

    plt.subplot(1, 6, 6)
    plt.imshow(flow)
    plt.axis('off')
    plt.title('flow')

In [ ]:
# Close the engine
xf_runner.close()